In [15]:
import pandas as pd

In [16]:
recall_df = pd.read_csv('ford_recalls_last.csv')
investigation_df = pd.read_csv('merged_2.csv')

In [17]:
recall_df.columns

Index(['Manufacturer', 'NHTSACampaignNumber', 'ReportReceivedDate',
       'Component', 'Summary', 'Consequence', 'Remedy', 'Notes', 'ModelYear',
       'Make', 'Model', 'parkIt', 'parkOutSide', 'overTheAirUpdate',
       'NHTSAActionNumber'],
      dtype='object')

In [18]:
investigation_df.columns

Index(['odiNumber', 'crash', 'fire', 'numberOfInjuries', 'numberOfDeaths',
       'dateOfIncident', 'dateComplaintFiled', 'vin', 'components',
       'summary_complaint', 'products', 'Model', 'ModelYear',
       'NHTSA_ACTION_NUMBER', 'MODEL', 'YEAR', 'component_name', 'open_date',
       'close_date', 'campaign_number', 'subject', 'summary_investigation',
       'similarity_score', 'recall_status'],
      dtype='object')

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
complaints_df = pd.read_csv("ford_complaints.csv")
investigations_df = pd.read_csv("investigation_with_recall_status.csv")

In [5]:
def preprocess_text(text):
    if pd.isnull(text):
        return ""
    return text.lower().strip()

complaints_df['Model'] = complaints_df['Model'].apply(preprocess_text)
complaints_df['ModelYear'] = complaints_df['ModelYear'].astype(str)

investigations_df['MODEL'] = investigations_df['MODEL'].apply(preprocess_text)
investigations_df['YEAR'] = investigations_df['YEAR'].astype(str)

In [6]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [7]:
complaint_embeddings = model.encode(complaints_df['summary'].fillna("").tolist(), convert_to_tensor=False)
investigation_embeddings = model.encode(investigations_df['SUMMARY'].fillna("").tolist(), convert_to_tensor=False)

complaints_df['summary_embedding'] = list(complaint_embeddings)
investigations_df['summary_embedding'] = list(investigation_embeddings)

In [8]:
merged_results = []

In [13]:
for _, complaint in complaints_df.iterrows():
    filtered_investigations = investigations_df[
        (investigations_df['MODEL'] == complaint['Model']) &
        (investigations_df['YEAR'] == complaint['ModelYear'])
    ]

    if filtered_investigations.empty:
        continue 

    similarities = cosine_similarity(
        [complaint['summary_embedding']],
        list(filtered_investigations['summary_embedding'])
    )[0]

    threshold = 0.7
    for idx, similarity in enumerate(similarities):
        if similarity >= threshold:
            investigation = filtered_investigations.iloc[idx]
            merged_results.append({
                'odiNumber': complaint['odiNumber'],
                'crash': complaint['crash'],
                'fire': complaint['fire'],
                'numberOfInjuries': complaint['numberOfInjuries'],
                'numberOfDeaths': complaint['numberOfDeaths'],
                'dateOfIncident': complaint['dateOfIncident'],
                'dateComplaintFiled': complaint['dateComplaintFiled'],
                'vin': complaint['vin'],
                'components': complaint['components'],
                'summary_complaint': complaint['summary'],
                'products': complaint['products'],
                'Model': complaint['Model'],
                'ModelYear': complaint['ModelYear'],
                'NHTSA_ACTION_NUMBER': investigation['NHTSA ACTION NUMBER'],
                'MODEL': investigation['MODEL'],
                'YEAR': investigation['YEAR'],
                'component_name': investigation['COMPNAME'],
                'open_date': investigation['ODATE'],
                'close_date': investigation['CDATE'],
                'campaign_number': investigation['CAMPNO'],
                'subject': investigation['SUBJECT'],
                'summary_investigation': investigation['SUMMARY'],
                'similarity_score': similarity,
                'recall_status': investigation['recall_status']
            })


In [14]:
merged_df = pd.DataFrame(merged_results)

merged_df.drop_duplicates(inplace=True)

merged_df.to_csv("merged_2.csv", index=False)

In [11]:
#assigning recall status and adding recall date to the merged data
import numpy as np
investigation_df = pd.read_csv('merged_2.csv')
recall_df = pd.read_csv('ford_recalls_last.csv')

def get_recall_info(row):
    match = recall_df[
        (recall_df['NHTSA_Campaign_Number'] == row['NHTSA_Campaign_Number']) | 
        (recall_df['NHTSA_ACTION_NUMBER'] == row['NHTSA_ACTION_NUMBER'])
    ]
    
    if not match.empty:
        return pd.Series([1, match['ReportReceivedDate'].values[0]])
    else:
        return pd.Series([0, -1])

investigation_df[['recall_status', 'ReportReceivedDate']] = investigation_df.apply(get_recall_info, axis=1)



In [14]:
investigation_df['dateOfIncident'] = pd.to_datetime(investigation_df['dateOfIncident'], format='%m/%d/%Y')
investigation_df['dateComplaintFiled'] = pd.to_datetime(investigation_df['dateComplaintFiled'], format='%m/%d/%Y')
investigation_df['open_date'] = pd.to_datetime(investigation_df['open_date'], format='%Y%m%d')
investigation_df['close_date'] = pd.to_datetime(investigation_df['close_date'], format='%Y%m%d')
investigation_df['ReportReceivedDate'] = pd.to_datetime(investigation_df['ReportReceivedDate'], format='%d/%m/%Y', errors='coerce')

#Calculate "incident_filing_lag" as the difference between "dateOfIncident" and "dateComplaintFiled"
investigation_df['incident_filing_lag'] = (investigation_df['dateComplaintFiled'] - investigation_df['dateOfIncident']).dt.days

# Calculate "days_taken_in_investigation" as the difference between "open_date" and "close_date"
investigation_df['days_taken_in_investigation'] = (investigation_df['close_date'] - investigation_df['open_date']).dt.days

#Calculate "days_taken_in_recall" as the difference between "dateComplaintFiled" and "ReportReceivedDate"
investigation_df['days_taken_in_recall'] = (investigation_df['ReportReceivedDate'] - investigation_df['dateComplaintFiled']).dt.days

In [15]:
investigation_df.to_csv('merged_4.csv', index=False)